# 1. Housekeeping

In [1]:
import ciao_contrib.runtool
from ciao_contrib.runtool import *
import pandas as pd
import numpy as np
from numpy import load
import astropy 
from astropy.table import Table
from astropy.io import fits
import os
import glob
import matplotlib.pyplot as plt
import ipywidgets as widgets

# List Folders Function
def list_folders(path):
    return [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    
# Specify path
path = '/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/'
folders = list_folders(path)
folder_list = []
for folder in folders:
    folder_list.append(folder)

# Select Set
set_widget = widgets.Dropdown(options=folder_list[:],value=folder_list[1],description='Set :',disabled=False); set_widget

FileNotFoundError: [Errno 2] No such file or directory: '/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/'

In [74]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Folder List
data_folder_path = f'/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-{set_id}/Data'
data_folders = sorted(list_folders(data_folder_path))
# Select Folder
folder_widget = widgets.Dropdown(options=data_folders[:],value=data_folders[1],description='Folder :',disabled=False); folder_widget

Dropdown(description='Folder :', index=1, options=('ra-0-5', 'ra-10-10p5', 'ra-100-110', 'ra-10p5-10p6', 'ra-1…

# 2. Source Region Filter Files

Incremental (preferred)

In [52]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Folder ID
folder_id = folder_widget.value
# File Loop
for event_filename in glob.iglob(f'{data_folder_path}/{folder_id}/acisf*regevt3.fits.gz'):
    obsid = int(event_filename.split('_')[0][-5:])
    try: 
        regionid = int(event_filename.split('_')[2][-4:])
    except: 
        regionid = int(event_filename.split('_')[3][-4:]) 
    region_filename = [region for region in glob.iglob(f'{data_folder_path}/{folder_id}/acisf*reg3.fits.gz') if str(obsid) in region and str(regionid) in region][0]
    print('Event Filename: ', event_filename)
    print('Region Filename: ', region_filename)
    filtered_filename = event_filename.replace(".fits", "_filtered.fits")
    print('Filtered Event Filename: ', region_filename)
    try:
        ciao_contrib.runtool.dmcopy(f'{event_filename}[sky=region({region_filename})]', filtered_filename)
    except OSError: 
        print(f'Filtered Event File {filtered_filename} already exists!')

print('DONE!')

Event Filename:  /Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf05404_000N020_r0033_regevt3.fits.gz
Region Filename:  /Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf05404_000N020_r0033_reg3.fits.gz
Filtered Event Filename:  /Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf05404_000N020_r0033_reg3.fits.gz
Event Filename:  /Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf08261_000N020_r0063_regevt3.fits.gz
Region Filename:  /Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf08261_000N020_r0063_reg3.fits.gz
Filtered Event Filename:  /Users/steven/Library/

Full Loop (alternative to incremental approach, not preferred)

In [ ]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Folder List
data_folder_path = f'/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-{set_id}/Data'
data_folders = list_folders(data_folder_path)
# Folder Loop
for data_folder_name in data_folders:
    # File Loop
    for event_filename in glob.iglob(f'{data_folder_path}/{data_folder_name}/acisf*regevt3*gz'):
        obsid = int(event_filename.split('_')[0][-5:])
        try: 
            regionid = int(event_filename.split('_')[2][-4:])
        except: 
            regionid = int(event_filename.split('_')[3][-4:]) 
        region_filename = [region for region in glob.iglob(f'{data_folder_path}/{data_folder_name}/acisf*reg3.fits.gz') if str(obsid) in region and str(regionid) in region][0]
        filtered_filename = event_filename.replace(".fits", "_filtered.fits")
        ciao_contrib.runtool.dmcopy(f'{event_filename}[sky=region({region_filename})]', filtered_filename)

        print(filtered_filename)

In [ ]:
# test1 = '/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-0-5/acisf11081_000N020_r0100_regevt3_filtered.fits.gz'
# test2 = '/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-0-5/acisf11081_000N020_r0100_regevt3.fits.gz'

# df1 = fits.open(test1)
# df2 = fits.open(test2)

# len(df1)

# with fits.open(test1) as hdul:
#     events1 = hdul["Events"].data
#     df_events1 = pd.DataFrame.from_records(Table(events1), columns=events1.columns.names)

# with fits.open(test2) as hdul:
#     events2 = hdul["Events"].data
#     df_events2 = pd.DataFrame.from_records(Table(events2), columns=events2.columns.names)

# print(len(df_events1))
# print(len(df_events2))

# 3. Load and Save Event File Table

Incremental

In [53]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Folder ID
folder_id = folder_widget.value
# Initialise dataframe list of all eventfiles
list_df_events = []
# File Loop
for filename in glob.iglob(f'{data_folder_path}/{folder_id}/acisf*regevt*filtered*gz'):
  print(filename)
  with fits.open(filename) as hdul:
      # Events
      events = hdul["Events"].data
      events_table = Table(events)
      events_cols = events.columns.names
      df_events = pd.DataFrame.from_records(events_table, columns=events_cols)
      df_events = df_events.sort_values(by=["time"])
      # GTI (Good Time Interval)
      gti = hdul["GTI"].data
      gti_table = Table(gti)
      gti_cols = gti.columns.names
      df_gti = pd.DataFrame.from_records(gti_table, columns=gti_cols)
      # Apply GTI Filter to Events
      gti_mask = np.zeros(len(df_events), dtype=bool)
      for i in range(len(df_gti)):
          start = df_gti.iloc[i]['START']
          stop = df_gti.iloc[i]['STOP']
          gti_mask |= (df_events["time"] >= start) & (df_events["time"] < stop)
      df_events = df_events[gti_mask]
      # Apply energy, pha, grade Filter to Events
      df_events = df_events[(df_events['pha']>40) & (df_events['grade']>=0) & (df_events['energy']>500) & (df_events['energy']<7000)]
      # Add obsid and region_id column (from filename)
      df_events["obsid"] = int(filename.split('_')[0][-5:])
      try: 
        df_events["region_id"] = int(filename.split('_')[2][-4:]) #need to add try except while looping 
      except: 
        df_events["region_id"] = int(filename.split('_')[3][-4:]) 
      # Append to dataframe list
      list_df_events.append(df_events)

# Combine dfs in dataframe list into one df and save in folder
df_eventfiles = pd.concat(list_df_events)
df_eventfiles.to_csv(f'{path}Set-{set_id}/eventfiles-{set_id}-{folder_id}.csv',index=False)

/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf01988_000N021_r0018_regevt3_filtered.fits.gz
/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf14436_001N020_r0067_regevt3_filtered.fits.gz
/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf05427_002N021_r0020_regevt3_filtered.fits.gz
/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf05402_000N020_r0015_regevt3_filtered.fits.gz
/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-Y/Data/ra-280-290/acisf00948_001N020_r0057_regevt3_filtered.fits.gz
/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Cod

Full Loop (alternative to incremental approach, not preferred)

In [44]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Initialise dataframe list of all eventfiles
list_df_events = []
# Folder List
data_folder_path = f'/Users/steven/Library/CloudStorage/OneDrive-ImperialCollegeLondon/01-27-2023-FYP-CfA/4-Code-Data/Chandra-Data/Set-{set_id}/Data'
data_folders = list_folders(data_folder_path)
# Folder Loop
for data_folder_name in data_folders:
  # File Loop
  for filename in glob.iglob(f'{data_folder_path}/{data_folder_name}/acisf*regevt*filtered*gz'):
    with fits.open(filename) as hdul:
        # Events
        events = hdul["Events"].data
        events_table = Table(events)
        events_cols = events.columns.names
        df_events = pd.DataFrame.from_records(events_table, columns=events_cols)
        df_events = df_events.sort_values(by=["time"])
        # GTI (Good Time Interval)
        gti = hdul["GTI"].data
        gti_table = Table(gti)
        gti_cols = gti.columns.names
        df_gti = pd.DataFrame.from_records(gti_table, columns=gti_cols)
        # Apply GTI Filter to Events
        gti_mask = np.zeros(len(df_events), dtype=bool)
        for i in range(len(df_gti)):
            start = df_gti.iloc[i]['START']
            stop = df_gti.iloc[i]['STOP']
            gti_mask |= (df_events["time"] >= start) & (df_events["time"] < stop)
        df_events = df_events[gti_mask]
        # Apply energy, pha, grade Filter to Events
        df_events = df_events[(df_events['pha']>40) & (df_events['grade']>=0) & (df_events['energy']>500) & (df_events['energy']<7000)]
        # Add obsid and region_id column (from filename)
        df_events["obsid"] = int(filename.split('_')[0][-5:])
        try: 
          df_events["region_id"] = int(filename.split('_')[2][-4:]) #need to add try except while looping 
        except: 
          df_events["region_id"] = int(filename.split('_')[3][-4:]) 
        # Append to dataframe list
        list_df_events.append(df_events)

# Combine dfs in dataframe list into one df and save in folder
df_eventfiles = pd.concat(list_df_events)
df_eventfiles.to_csv(f'{path}Set-{set_id}/eventfiles-{set_id}.csv',index=False)

# 3. Concatenate Eventfiles

42


In [62]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Folder ID
folder_id = folder_widget.value
# Load Eventfiles Table
df_events_list=[]
for evtfile_name in os.listdir(f'{path}Set-{set_id}/'):
    if evtfile_name.startswith(f'eventfiles-{set_id}-ra'):
        df_evt = pd.read_csv(f'{path}Set-{set_id}/{evtfile_name}')
        df_events_list.append(df_evt)
        count = count + 1
df_evt = pd.concat(df_events_list)
df_evt.to_csv(f'{path}Set-{set_id}/eventfiles-{set_id}.csv',index=False)
print('DONE!')

Progress: 1 / 42
Progress: 2 / 42
Progress: 3 / 42
Progress: 4 / 42
Progress: 5 / 42
Progress: 6 / 42
Progress: 7 / 42
Progress: 8 / 42
Progress: 9 / 42
Progress: 10 / 42
Progress: 11 / 42
Progress: 12 / 42
Progress: 13 / 42
Progress: 14 / 42
Progress: 15 / 42
Progress: 16 / 42
Progress: 17 / 42
Progress: 18 / 42
Progress: 19 / 42
Progress: 20 / 42
Progress: 21 / 42
Progress: 22 / 42
Progress: 23 / 42
Progress: 24 / 42
Progress: 25 / 42
Progress: 26 / 42
Progress: 27 / 42
Progress: 28 / 42
Progress: 29 / 42
Progress: 30 / 42
Progress: 31 / 42
Progress: 32 / 42
Progress: 33 / 42
Progress: 34 / 42
Progress: 35 / 42
Progress: 36 / 42
Progress: 37 / 42
Progress: 38 / 42
Progress: 39 / 42
Progress: 40 / 42
Progress: 41 / 42
Progress: 42 / 42
DONE!


# 4. Generate Input

In [10]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Load Properties Table
df_prop = pd.read_csv(f'{path}Set-{set_id}/properties-{set_id}.csv')
# Load Properties Table
df_events = pd.read_csv(f'{path}Set-{set_id}/eventfiles-{set_id}.csv')
# Extract important labels and input columns
df_labels = df_prop[['obsid','region_id','cnts_aper_b','cnts_aperbkg_b','src_cnts_aper_b','flux_aper_b','hard_hm','hard_hs','hard_ms','var_prob_b','var_prob_h','var_prob_m','var_prob_s']]
df_input = df_events[['obsid','region_id','time','energy','chipx','chipy']]
# Make Sure Eventfile Table only includes information contained in Properties Table and vice versa
df_labels['obsreg_id'] = df_labels['obsid'].astype(str) + '_' + df_labels['region_id'].astype(str)
df_input['obsreg_id'] = df_input['obsid'].astype(str) + '_' + df_input['region_id'].astype(str)
df_labels_group = df_labels.drop(columns=['obsid', 'region_id'])
df_input_group = df_input.drop(columns=['obsid', 'region_id'])
df_labels_group = df_labels.drop(columns=['obsid', 'region_id'])
df_input_group = df_input.drop(columns=['obsid', 'region_id'])
df_labels_group = df_labels_group.groupby('obsreg_id')
df_input_group = df_input_group.groupby('obsreg_id')


print(len(df_input['obsreg_id'].unique()))
print(len(df_labels['obsreg_id'].unique()))

print(df_input_group.ngroups)
print(df_labels_group.ngroups)

# ID filter
labels_ids = list(df_labels_group.groups.keys())
input_ids = list(df_input_group.groups.keys())
df_labels_group2 = df_labels_group.filter(lambda x: x.name in input_ids)
df_input_group2 = df_input_group.filter(lambda x: x.name in labels_ids)

df_labels2 = df_labels[df_labels['obsreg_id'].isin(df_input['obsreg_id'].unique())]
df_input2 = df_input[df_input['obsreg_id'].isin(df_labels['obsreg_id'].unique())]
# Save ungrouped tables
df_labels.to_csv(f'{path}Set-{set_id}/labels-{set_id}.csv',index=False)
df_input.to_csv(f'{path}Set-{set_id}/inputs-{set_id}.csv',index=False)
df_labels2.to_csv(f'{path}Set-{set_id}/labels2-{set_id}.csv',index=False)
df_input2.to_csv(f'{path}Set-{set_id}/inputs2-{set_id}.csv',index=False)
# Save grouped tables
df_labels_group.to_csv(f'{path}Set-{set_id}/labelsgroup-{set_id}.csv',index=False)
df_input_group.to_csv(f'{path}Set-{set_id}/inputsgroup-{set_id}.csv',index=False)
# Save grouped tables
df_labels_group2.to_csv(f'{path}Set-{set_id}/labelsgroupsmall-{set_id}.csv',index=False)
df_input_group2.to_csv(f'{path}Set-{set_id}/inputsgroupsmall-{set_id}.csv',index=False)

/var/folders/pr/pzfn3sxs66jfxwx55jdxqrd00000gn/T/ipykernel_706/1870246970.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels['obsreg_id'] = df_labels['obsid'].astype(str) + '_' + df_labels['region_id'].astype(str)
/var/folders/pr/pzfn3sxs66jfxwx55jdxqrd00000gn/T/ipykernel_706/1870246970.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['obsreg_id'] = df_input['obsid'].astype(str) + '_' + df_input['region_id'].astype(str)


179372
119023
179372
119023


AttributeError: 'DataFrameGroupBy' object has no attribute 'to_csv'

In [ ]:
main_list = list(set(df_input['obsreg_id'].unique()) - set(df_labels['obsreg_id'].unique()))
main_list2 = list(set(df_labels['obsreg_id'].unique()) - set(df_input['obsreg_id'].unique()))

In [8]:

print(main_list[0])
print(main_list2[0])

print(len(df_input2['obsreg_id'].unique()))
print(len(df_labels2['obsreg_id'].unique()))



14368_438
9093_30
85056
85056


In [ ]:
# Set ID
set_id = set_widget.value.split("-")[1]
# Load Properties Table
df_prop = pd.read_csv(f'{path}Set-{set_id}/properties-{set_id}.csv')
# Load Properties Table
df_events = pd.read_csv(f'{path}Set-{set_id}/eventfiles-{set_id}.csv')
# Extract important labels and input columns
df_labels = df_prop[['obsid','region_id','cnts_aper_b','cnts_aperbkg_b','src_cnts_aper_b','flux_aper_b','hard_hm','hard_hs','hard_ms','var_prob_b','var_prob_h','var_prob_m','var_prob_s']]
df_input = df_events[['obsid','region_id','time','energy','chipx','chipy']]
# Make Sure Eventfile Table only includes information contained in Properties Table and vice versa
df_labels['obsreg_id'] = df_labels['obsid'].astype(str) + '_' + df_labels['region_id'].astype(str)
df_input['obsreg_id'] = df_input['obsid'].astype(str) + '_' + df_input['region_id'].astype(str)

print(len(df_labels['obsreg_id'].unique()))
print(len(df_input['obsreg_id'].unique()))


print("LABELS")
print(sorted(df_labels['obsreg_id'].unique()))
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print("INPUTS")
print(sorted(df_input['obsreg_id'].unique()))


